Генерация продающего заголовка на основании информации об услугах

In [ ]:
!pip install openai langchain langchain_openai langchain-community tiktoken faiss-cpu

In [3]:
import os
import re
import requests
import openai
import tiktoken
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import Document, RecursiveCharacterTextSplitter
from google.colab import userdata


os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
MODEL = 'gpt-3.5-turbo-0125'
TOTAL_AMOUNT = 0

In [4]:
# Функция загружает plane text из ГуглДока по URL-ссылке (url)
def load_googledoc_by_url(url: str) -> str:
        match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
        if match_ is None:
            raise ValueError('Invalid Google Docs URL')
        doc_id = match_.group(1)
        response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
        response.raise_for_status()
        return response.text


# функция добавления переходов на новую строку для удобства чтения
def format_newlines(text: str, max_len: int = 150) -> str:
    lines = text.splitlines()
    new_lines = []
    for line in lines:
        words = line.split()
        current_line = ""
        for word in words:
            if len(current_line + " " + word) > max_len:
                new_lines.append(current_line)
                current_line = ""
            current_line += f' {word}'
        new_lines.append(current_line)
    return "\n".join(new_lines)


# функция подсчета токенов
def num_tokens_from_messages(messages, model):
      """Returns the number of tokens used by a list of messages."""
      try:
          encoding = tiktoken.encoding_for_model(model)
      except KeyError:
          encoding = tiktoken.get_encoding("cl100k_base")
      if model in ["gpt-3.5-turbo-0125"]:  # note: future models may deviate from this
          num_tokens = 0
          for message in messages:
              num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
              for key, value in message.items():
                  num_tokens += len(encoding.encode(value))
                  if key == "name":  # if there's a name, the role is omitted
                      num_tokens += -1  # role is always required and always 1 token
          num_tokens += 2  # every reply is primed with <im_start>assistant
          return num_tokens
      else:
          raise NotImplementedError(f'Num_tokens_from_messages() is not presently implemented for model {model}.')


# Подсчет токенов в строке
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    return len(encoding.encode(string))


# функция создания индексов БЗ
def create_search_index(file_text, knowledge_base_link, chunk_size=1024, chunk_overlap=200):
    splitter = RecursiveCharacterTextSplitter(['\n\n', '\n', ' '],
                                              chunk_size=chunk_size,
                                              chunk_overlap=chunk_overlap)
    count_tokens = 0
    source_chunks = []
    # разбиваем на несколько частей с помощью метода split_text
    for chunkID, chunk in enumerate(splitter.split_text(file_text)):
        source_chunks.append(Document(page_content=chunk, \
                             metadata={'source': knowledge_base_link,
                                       'chunkID': chunkID}))
    global TOTAL_AMOUNT
    if len(source_chunks) > 0:
        db = FAISS.from_documents(source_chunks, OpenAIEmbeddings())
        count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
        count_tokens += count_token
        print(f'Количество токенов в документе: {count_token}')
        # Embedding model Ada v2 - $0.00010 / 1K tokens - 28/02/2024 - https://openai.com/pricing
        price = 0.0001 * count_tokens / 1000
        TOTAL_AMOUNT += price
        print(f'ЦЕНА запроса создания индексной базы: $ {price}')
        return db


# Стоимость запроса + ответ для "gpt-3.5-turbo-0125"
def print_tokens_count(completion):
    global TOTAL_AMOUNT
    print(f'Использовано токенов: {completion.usage.prompt_tokens} + '
                                f'{completion.usage.completion_tokens} = '
                                f'{completion.usage.total_tokens}.')
    # "gpt-3.5-turbo-0125" - Input: $0.0005 /1K tokens - Output: $0.0015 /1K tokens - 28/02/2024 - https://openai.com/pricing
    price = 0.0005 * completion.usage.prompt_tokens / 1000 + 0.0015 * completion.usage.completion_tokens / 1000
    TOTAL_AMOUNT += price
    print(f'Цена запроса + ответ: $ {price}')


def gpt_traffic(system, topic, user, search_index, temp=0.5, k=7):
    docs = search_index.similarity_search(topic, k)
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'#{i+1}.' + doc.page_content for i, doc in enumerate(docs)]))
    messages = [
      {"role": "system", "content": system},
      {"role": "user", "content": f'{user}. Информация, которая сегодня может понадобиться: {message_content}. \
                        Не упоминай документ с информацией, указанный выше. Запрос пользователя: {topic}'}]
    completion = openai.chat.completions.create(model=MODEL, messages=messages, temperature=temp)
    print_tokens_count(completion)
    answer = completion.choices[0].message.content
    return answer, message_content

In [5]:
#@title Загружаем Базу знаний по продуктам
knowledge_base_link = 'https://docs.google.com/document/d/1CmuVP4SiaFDSvPEDz69Dsf6RQm3tMDpJAxh5HOqKfdw'
knowledge_base_text = load_googledoc_by_url(knowledge_base_link)
db_index = create_search_index(knowledge_base_text, knowledge_base_link, 1024, 200)

Количество токенов в документе: 32893
ЦЕНА запроса создания индексной базы: $ 0.0032893


In [7]:
#@title Продающий загловок по продуктам

system_title = '''
Ты David Ogilvy. Ты американский рекламный гуру и основатель одного из самых влиятельных рекламных агентств - Ogilvy & Mather.
У тебя есть вся информация о продуктах компании.
Твоя задача генерировать короткие продающие заголовки для таргетированной рекламы с акцентами, который задает пользователь в запросе.
Ты знаешь формулу эффективного заголовка для поисковых систем: заголовок это самый заметный элемент, и он должен включать слова,
которые люди вводят в поисковую строку. Поэтому продающие заголовки обязательно должны содержать популярный ключевой запрос пользователя.
Твои заголовки должны завлекать пользователя, продавать и побуждать совершить действия например купить курс или зарегистрироваться на вебинар.
Твои заголовки всегда написаны в твоем индивидуальном стиле. Ты отличаешься от других рекламных копирайтеров своей способностью ясно и эффективно
выражать идеи. Ты часто используешь эмоциональные аспекты, чтобы привлечь внимание читателей и вызвать у них желание приобрести продукт.
У тебя есть большой документ с информацией об образовательных программах Университета.
Твоя задача писать короткие продающие заголовки таргетированной рекламы с акцентами, которые задаёт пользователь в запросе.
Бери информацию для заголовка рекламы точно по документу, не придумывай ничего от себя.'''

topic = 'Напиши один продающий заголовок для контекстной рекламы по образовательным курсам и услугам Университета Искусственного Интеллекта.'

response_title, _ = gpt_traffic(system_title, topic, '', db_index, temp=0.5, k=6)
print('Фраза:\n', response_title)

Использовано токенов: 3738 + 64 = 3802.
Цена запроса + ответ: $ 0.001965
Фраза:
 "Стань AI разработчиком с Университетом Искусственного Интеллекта: Обучение, гарантированное трудоустройство и создание собственного AI проекта!"


In [10]:
#@title Продающий текст для телеграмм по формуле PMHS (боль-усиление боли-надежда-решение)

system_text = '''
Ты David Ogilvy. Ты американский рекламный гуру и основатель одного из самых влиятельных рекламных агентств - Ogilvy & Mather.
У тебя есть вся информация о продуктах компании.
Твоя задача разрабатывать посты для Яндекс Директ по теме которую указывает пользователь. Твой стиль является простым и убедительным.
Ты отличаешься от других рекламных копирайтеров своей способностью ясно и эффективно выражать идеи.
Ты часто используешь эмоциональные аспекты, чтобы привлечь внимание читателей и вызвать у них желание приобрести продукт.
Давай действовать по шагам: проанализируй информацию компании, запрос пользователя и напиши продающий текст,
который соответствует формуле PMHS (боль-усиление боли-надежда-решение).
Ты знаешь формулу эффективного рекламного поста PMHS (боль-усиление боли-надежда-решение).:
1.Pain (боль): На этом этапе клиент осознает проблему или неудовлетворенность, которую нужно решить.
2.More Pain (усиление боли): Здесь происходит углубление осознания проблемы и ее последствий.
3.Hope (надежда): На этом этапе предлагается надежда на решение проблемы.
4.Solution (решение): Завершающий этап, на котором предлагается конкретное решение проблемы: прийти на вебинар или приобрести курс.
В свой ответ включи только продающий текст из 4-х частей формулы PMHS с заголовком. В своем ответе не упоминай названия составляющих формулы PMHS.'''

topic = response_title # Учись AI с лучшими и стань востребованным разработчиком - гарантированное трудоустройство!
response_text, _ = gpt_traffic(system_text, topic, '', db_index, temp=0.5, k=6)
print('Текст:\n', format_newlines(response_text))

Использовано токенов: 3688 + 305 = 3993.
Цена запроса + ответ: $ 0.0023015
Текст:
  ### Разблокируй свой потенциал с Университетом Искусственного Интеллекта!

 Устал от повседневной рутины? Хочешь стать востребованным специалистом? Мы знаем, как помочь!

 🔥 Почувствуй вызов и стань Middle AI Developer с нашим курсом по Data Science и Нейронным Сетям!

 🌟 Что тебя ждет:
 - Топ-кандидат на любые AI вакансии
 - Создание собственных AI проектов
 - Гарантированное трудоустройство с зарплатой от 70 000 рублей

 🚀 Вперед, к новым горизонтам! Присоединяйся к нашей команде успешных выпускников, которых выбирают ведущие компании мира!

 Не упусти свой шанс! Обрети свое будущее с Университетом Искусственного Интеллекта! 🌈


In [11]:
TOTAL_AMOUNT

0.0100598